# Day4 「Pythonテトリス」の作り方
テトリスは、ミノが置かれていない状態を「0」、I字型のミノを「1」、L字型のミノを「2」・・・と数字で呼ぶことにして、<br>
`0`から`7`までの数字が`22`段 × `10`列 = `220`個並んだ「NumPy配列」によって表現されている、とDay2で述べました。<br><br>

テトリスのコードの仕組みを把握するポイントは、
<b>

1. ミノの形・回転・移動・落下というルールを、「NumPy配列」によってどうやって表現できる？
1. ミノをどの向きでどこに落とすか、どうやって決めればいい？

</b>
という2点です。<br>
文法学習が終わって中盤戦に入るDay4では、本番で使われるコードでこれらの実装を確認し、実際に挙動を試してみることでその理解を深めます。<br>

## 0. チュートリアルのゴール
<b>ルールベースの場合には`block_controller.py`を、AIの場合には`block_controller_train.py`を書き換えられる</b>ようになることが、学習のゴールです。

### 文法学習の補足
Day3までの文法学習では、1つの`.py`ファイルや`.ipynb`ファイルでコードを書いてきました。<br>
しかし、このテトリスなどのゲームや実際の製品でコードが数百〜数万行にまで膨れてくると、1つのファイルでは行数が多くて読みづらくなってしまいます。<br>
そこでPythonでは、複数のファイルにコードを分割することができます。<br>
実は今まで「おまじない」として使っていた`import`文の正体は、別のファイルにあるコードを使うための文法でした。<br><br>

試しに下のセルを実行してみてください。<br>
このセルにはクラスの定義文はおろか`print`文もないのに、いかにもクラスの関数を使用しているっぽい1行が書かれている上に、ある一文が出力されます。<br><br>

これは、同じにフォルダに保存された`import_tutorial.py`という別のファイルにあるコードを実行しているからです。<br>
`import_tutorial.py`には、文字を表示する機能である関数`printChar()`と、その文字を保存した変数`self.s`を定義する「コンストラクタ」を持つ、`ImportTutorial()`クラスが定義されています。<br>
クラスの持つ変数や機能を使うためにはクラスの「インスタンス」を作る必要があり、それは13行目で`IMPORT_TUTORIAL`という名前でなされています。<br>
(クラスとは料理のレシピのように、材料という「変数」と調理手順という「関数」を決めたもので、実際にごはんを食べるにはレシピに従って調理する「インスタンス化」が必要でした)<br><br>

テトリスの本番コードでも同じように、別のファイルにあるクラスが使われるときには<br>
`from (別のファイル名から拡張子「.py」を除いたもの) import (インスタンスを作った変数の名前)`<br>
とコードの冒頭に書かれていて、「別のファイルから変数を自分のコードに取り込できなさい」という命令を表しています。

In [ ]:
from import_tutorial import IMPORT_TUTORIAL
IMPORT_TUTORIAL.printChar()

### 本番コードのファイル構成
複数ファイルにコードが分割されていても実行する方法があることを知った上で、テトリスの本番コードにおける`.py`ファイルの場所と役割を、<b><a href="https://github.com/seigot/tetris/tree/master#%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E6%A7%8B%E6%88%90">GitHubのREADMEファイル</a></b>で確認しておきましょう。<br><br>

テトリスを実行するときコマンドプロンプトやターミナルに<br>
`python start.py`<br>
という一行を入力して`Enter`キーを押していますが、これは「`start.py`の中身を実行せよ」という命令を意味していました。<br>
READMEの図は`start.py`を起点に、複数のファイルにまたがったクラスを呼び出している繋がりを表しています。<br><br>

上で述べた

- ミノの形・回転・移動・落下というルール

やゲーム画面の表示は、`game_manager`フォルダにある`game_manager.py`という<b>ゲーム管理用プログラム</b>と、`board_manager.py`という<b>ルール管理用プログラム</b>に、

- ミノをどの向きでどこに落とすか

という、皆さんに作っていただきたいミノの落とし方の作戦は、同じく`game_manager`フォルダにある`block_controller.py`、<br>
もしくは`game_manager`フォルダの中の`machne_learning`フォルダにある`block_controller_train.py`という<b>ブロック操作用プログラム</b>に、それぞれ実装されています。<br><br>

`block_controller.py`もしくは`block_controller_train.py`の中でも、<b>関数`GetNextMove()`が落とし方の作戦を計算している</b>部分です。<br>
本チュートリアルは、この関数で登場するミノの落とし方や作戦の計算方法を学ぶことが目的です。<br>
詳しい作りや流れはさておき、<span style="color: red; ">**実際にコードを読むときにはチュートリアル以上の内容は読み込まなくて問題ない**</span>ように、必要な部分だけを本番コードから抜き出して作っています。

<div align="center">
<img src="img/tetris_mermaid.png" width="500">
</div>

### 演習準備
今回のチュートリアルでも必要なライブラリの`import`や、視覚的に分かりやすく表現するための関数`visualize_block`の定義を、予めしておきます。<br>
<span style="color: red; ">**関数の中身や使い方は本筋と関係ないので、読み飛ばして実行だけしてください。**</span>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def visualize_block(board, shape=(22,10), size=0.5):
    board = board.reshape(shape)
    block_array = []
    fig, ax = plt.subplots()
    fig.set_figwidth(shape[0] * size)
    fig.set_figheight(shape[1] * size)
    for i in range(shape[0]):
        row = []
        for j in range(shape[1]):
            c = colors[int(board[i][j])]
            row.append(c)
        block_array.append(row)
    block_array = np.array(block_array)

    ax.set_xticks(np.arange(-0.5, 10, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, 23, 1.0), minor=True)
    ax.set_xticks(np.arange(-0.5, 10,10))
    ax.set_yticks(np.arange(-0.5, 23, 3))
    ax.grid(which='minor', color='black', linestyle='-', linewidth=0.5)
    ax.grid(which='major', color='black', linestyle='-', linewidth=0.5)
    ax.imshow(block_array)

colors = [[255, 255, 255],
          [255,   0,   0],
          [  0, 255,   0],
          [255, 255,   0], 
          [218, 179,   0],
          [247, 171, 166],
          [  0,   0, 255],
          [255, 255,   0]]

- - -

## 1. ミノの形と機能を定義しよう
まずは、落ちてくるブロックである「ミノ」の形と機能をコードで表現するため、実装したいミノの仕様について<b><a href="https://github.com/seigot/tetris/blob/master/doc/files/block_controller.md#%E3%83%96%E3%83%AD%E3%83%83%E3%82%AF%E6%83%85%E5%A0%B1">GitHubのブロック操作用プログラムのブロック情報</a></b>を確認しましょう。<br>
ミノの形というテトリスのルールに関する実装は、`board_manager.py`に記述されています。

### ブロックの定義
初めに述べたようにテトリスの盤面は、ミノに番号を付けた数字の並び「配列」として表現するのでした。<br>
4つの四角が組み合わさったミノの形は7種類あるので、それぞれ

1. `shapeI`：I字
1. `shepeL`：L字
1. `shapeJ`：J字
1. `shapeT`：T字
1. `shapeO`：正方形
1. `shapeS`：S字
1. `shapeZ`：Z字

という番号をその変数名に保存しておくことにしています。さらに<br>

0. ミノが置かれていない状態

を合わせて、`0`から`7`までの数字が配列には並ぶことになります。

### ブロック操作の基準点
ミノの形は、4つの四角が位置する座標`(x, y)`によって表現しています。<br>
ミノの中で基準となる四角をひとつ選んでその座標を`(0, 0)`とし、他の3つの四角はその基準点からの相対的な位置関係で表します。<br>
ここでテトリスにおける座標とは、<a href="https://github.com/seigot/tetris/blob/master/doc/files/block_controller.md#%E3%83%95%E3%82%A3%E3%83%BC%E3%83%AB%E3%83%89%E6%83%85%E5%A0%B1"><b>画面左上のマスを原点として右方向を`x`軸、下方向を`y`軸の正の向きとしている</b></a>ことに注意してください。<br><br>

例えばI字の形をしたミノ`shapeI`は、画面に初めて現れるときには縦 (文字通りIの形)としています。<br>
上から2つ目の四角を基準点`(0, 0)`とすれば、画面の下方向に`y`座標の正の向きを設定しているので、基準点の1つ上にある一番上の四角は`(0, -1)`という位置関係で表すことにします。<br>
同様にして、下から2つ目の四角は`(0, 1)`、一番下の四角は`(0, 2)`と表すことにします。<br><br>

コードでは、この4つの四角の座標を順に並べた「タプル」というデータの並びによって実装しています。<br>
書き方が違うだけで扱いは、数字が並んだ「配列」とほぼ変わりません。<br>
数字の代わりに、数字を2つ並べたセットが並んでいるだけだ、という認識で問題ありません。<br><br>

変数`shapeCoord`という「タプル」には、1要素目に`shapeI`の座標4つ、2要素目に`shapeL`の座標4つ、・・・というように7要素目の`shapeZ`まで座標の組を保存しておきます。<br>
例えば`shapeI`の4つの座標`(0, -1)`、`(0, 0)`, `(0, 1)`, `(0, 2)`という値を使いたいときには、`shapeCoord[1]`と記述することでこの4つの値だけを取り出すことができます。<br><br>

一度、ここまでの実装を確認してみましょう。<br>
ミノの種類と形という「状態」は、`board_manager.py`の`Shape`クラスに記述されています。<br>
下のセルを実行してください。

In [ ]:
class Shape(object):
    ###################
    # ブロックの定義
    ###################
    shapeNone = 0
    shapeI = 1
    shapeL = 2
    shapeJ = 3
    shapeT = 4
    shapeO = 5
    shapeS = 6
    shapeZ = 7

    ###################
    # ブロック操作の基準点
    ###################
    shapeCoord = (
        ((0, 0), (0, 0), (0, 0), (0, 0)),
        ((0, -1), (0, 0), (0, 1), (0, 2)),
        ((0, -1), (0, 0), (0, 1), (1, 1)),
        ((0, -1), (0, 0), (0, 1), (-1, 1)),
        ((0, -1), (0, 0), (0, 1), (1, 0)),
        ((0, 0), (0, -1), (1, 0), (1, -1)),
        ((0, 0), (0, -1), (-1, 0), (1, -1)),
        ((0, 0), (0, -1), (1, 0), (-1, -1))
    )

    ###################
    # コンストラクタ
    # 引数shape: Shapeクラスには全てのミノの情報が保存されているので、使いたいミノの番号を初めに指定する
    ###################
    def __init__(self, shape=0):
        self.shape = shape

### ブロックの回転
ミノはその種類と形という「状態」の他に、回転するという「機能」を持っています。<br>
すなわち、ミノが回転したときの座標4つがどの値に変わるかを計算してくれる「機能」です。<br><br>

ミノは四角形で構成されているので、ミノを4回転させれば必ず回転前の形と一致します。<br>
しかし`shapeI`のように2回転で元に戻るミノもあれば、`shapeO`のように回転させても形の変わらないミノもあります。<br>
回転させた形のパターンと座標は、<a href="https://github.com/seigot/tetris/blob/master/doc/files/block_controller.md#%E3%83%96%E3%83%AD%E3%83%83%E3%82%AF%E6%93%8D%E4%BD%9C%E3%81%AE%E5%9F%BA%E6%BA%96%E7%82%B9">形状毎の基準点とそれ以外の座標の表</a></b>にまとめてあります。<br><br>

4パターンのミノの向きを引数として与えると、その座標を返してくれる関数`getTotatedOffsets`が`Shape`クラスには実装されています。<br>
(本チュートリアルでは説明の便宜上、先ほど実行したセルで定義した`Shape`クラスを継承して、同じ名前の`Shape`クラスを作成しています)<br><br>

ミノの向きは、

0. 回転させない (`shapeO`は回転できない)
1. 1回転
2. 2回転 (`shapeI`・`shapeZ`・`shapeS`はこの時点で元に戻る)
3. 3回転 (`shapeI`・`shapeZ`・`shapeS`は1回転と同じ)

という数字で、引数である変数`direction`に与えられます。<br>
`return`の後ろに続く文は、回転させた座標の値を4つまとめて返してくれる数式です。<br>
例えば1回転させた場合には、`y`座標の値にマイナスをつけた上で`x`座標と`y`座標を入れ換える、という計算をしています。<br>
(Pythonに自信のある方は、「内包表記」で検索してみてください)<br><br>

では、2回転させたら座標はどのように計算できるでしょうか？<br>
「★」には、`x`・`-x`・`y`・`-y`のいずれかが入ります。<br><br>

穴埋めをしていただきたい不完全な箇所には、前回と同様に<br>
`#------------ここから--------------`
<br><br>
`#------------ここまで--------------`
<br>
を記載してあります。<br>
<span style="color: red; ">**下のセルの「★」4ヶ所を正しく埋めた上で、実行してください。**</span>

In [ ]:
class Shape(Shape):
    ###################
    # テトリミノ形状を回転した座標を返す
    # direction: テトリミノ回転方向
    ###################
    def getRotatedOffsets(self, direction):
        
        # 指定したミノの番号「self.shape」によって、全てのミノの位置関係が保存された変数「shapeCoord」から、指定したミノの座標4点だけを取り出す
        tmpCoords = Shape.shapeCoord[self.shape]
        
        # 回転させない (「shapeO」は回転できない)
        if direction == 0 or self.shape == Shape.shapeO:
            return ((x, y) for x, y in tmpCoords)

        # 1回転
        if direction == 1:
            return ((-y, x) for x, y in tmpCoords)

        # 2回転 (「shapeI」「shapeZ」「shapeS」はこの時点で元に戻る)
        if direction == 2:
            if self.shape in (Shape.shapeI, Shape.shapeZ, Shape.shapeS):
                #------------ここから--------------
                return ((★, ★) for x, y in tmpCoords)
            else:
                return ((★, ★) for x, y in tmpCoords)
                #------------ここまで--------------

        # 3回転 (「shapeI」「shapeZ」「shapeS」は1回転と同じ)
        if direction == 3:
            if self.shape in (Shape.shapeI, Shape.shapeZ, Shape.shapeS):
                return ((-y, x) for x, y in tmpCoords)
            else:
                return ((y, -x) for x, y in tmpCoords)

お試しとして、1つ下のセルに好きなミノの番号を入力して実行し、続けて2つ目のセルも実行してみてください。<br>
表と同じ座標が4つ出力されていれば、穴埋めは成功しています。<br>

In [ ]:
#------------ここから--------------
minoNo = 1
#------------ここまで--------------

In [ ]:
minoExample = Shape(minoNo)
for i in minoExample.getRotatedOffsets(0):
    print(i)

### ブロックの配置
ミノは、基準点と相対的に定められた座標を用いて定義される位置関係を、実際に盤面に配置するという「機能」も持っています。<br>
配置するときには、「基準点をどの座標に配置するか」「どの向きで配置するか」を決めてあげればよく、同じく`Shape`クラスの関数`getCoords`で定義されています。<br>
`return`の後ろに続く文は、関数`getRotatedOffsets`と似ているように、4つの位置関係に実際の座標を足している式です。<br><br>

また、ミノを配置する上で`22`段 × `10`列の盤面内に収まっているかを判定するために使う、ミノの4つの座標の範囲を調べる関数`getBoundingOffsets`も用意されています。<br>
基準となる値を`0`に設定しておいて`for`文である座標の値とこの基準値を比較し、もし`0`より小さければ最小値を更新する、さらに更新した値と次の座標に値を比較する・・・という処理を行っています。

In [ ]:
class Shape(Shape):
    ###################
    # direction (回転状態)のテトリミノ座標配列を取得し、それをx,yに配置した場合の座標配列を返す
    ###################
    def getCoords(self, direction, x, y):
        return ((x + xx, y + yy) for xx, yy in self.getRotatedOffsets(direction))

    ###################
    # テトリミノが原点から x,y 両方向に最大何マス占有するのか返す
    ###################
    def getBoundingOffsets(self, direction):
        # テトリミノ形状を回転した座標を返す
        tmpCoords = self.getRotatedOffsets(direction)
        # 
        minX, maxX, minY, maxY = 0, 0, 0, 0
        for x, y in tmpCoords:
            if minX > x:
                minX = x
            if maxX < x:
                maxX = x
            if minY > y:
                minY = y
            if maxY < y:
                maxY = y
        return (minX, maxX, minY, maxY)

以上が、ミノの種類や形という「状態」と回転や配置という「機能」を定義したクラス`Shape`です。<br>
そしてゲームを管理する`game_manager.py`で、<span style="color: red; ">**落ちてくる予定のミノやホールドしているミノの情報を、Shapeクラスのインスタンスを用いて保存しています。**</span><br><br>

ここで、`game_manager.py`の実装を確認しておきましょう。<br>
関数`getGameStatus`という、実行された時点でのゲームの情報を`status`に格納する処理があり、その中に落ちてくる予定のミノの情報が含まれています。<br>
下のセルはこの関数の中から、次に落ちてくる予定のミノやその他のゲームのルールを抜き出してきたものです<br><br>

この関数は同じく`game_manager.py`の関数`timerEvent`で用いられており、ゲームの情報は変数`GameStatus`に保存されます。<br>
そして、皆さんに実装していただく`block_controller.py`で定義された関数`GetNextMove`では、これから落ちてくる予定のミノをどのように配置するかという作戦を計算するために、使われています。<br><br>

たくさん種類のあるゲームの情報を保存するために、ここでは「辞書型」という特殊な配列を使っています。<br>
これは単に数字を並べるのではなく、その数字が何を意味するのかという「目次」を付けてあげることのできる配列です。<br>
NumPy配列から数字を取り出すときには`[要素の番号]`という書き方をしていましたが、「辞書型」では`["目次の文字"]`という書き方をします。<br>
例えて言うなら、変数`GameStatus`の中で今盤面に落ちてきているミノの情報は、`block_info`という章の、`currentShape`という節の、`class`という項に記されています。<br><br>

なお、この関数の引数も`game_manager.py`での実装から本チュートリアル向けに改造しています。

In [ ]:
def getGameStatus(No):
    status = {
                # 盤面そのものの情報
                "field_info":
                {
                    # 盤面の横幅
                    "width": "none",
                    # 盤面の縦幅
                    "height": "none",
                },

                # 落ちてくる予定のミノの情報
                "block_info":
                {
                    # 今盤面に落ちてきているミノ
                    "currentShape":
                    {
                        # ミノのクラス「Shape」のインスタンス
                        "class": "none",
                    }
                },
             }

    ## board
    status["field_info"]["width"]  = 10
    status["field_info"]["height"] = 22

    ### current shape
    currentShapeClass = Shape(No)
    status["block_info"]["currentShape"]["class"] = currentShapeClass

    return status


- - -

## 2. 定義したミノを動かしてみよう
ゲーム管理用プログラム`game_manager.py`とルール管理用プログラム`board_manager.py`のおさらいと、ブロック操作用プログラム`block_controller.py`の理解のために、<br>
ここまで実装してきた`Shape`クラスと関数`getGameStatus`を使って例題を解いてみましょう。<br><br>

`block_controller.py`では、<b>いま盤面に現れたミノに対して全ての配置パターンを試してみる</b>、という処理を行なっています。<br>
下のセルに定義している関数`getSearchXRange`は、ある方向に回転したミノがどれだけ横移動できるかを計算する関数です。<br><br>

ミノを配置するときには、「基準点をどの座標に配置するか」を決めてあげる必要がある、と上で説明しました。<br>
例えば横向きに回転させた`shapeI` (伸ばし棒の形)は、<a href="https://github.com/seigot/tetris/blob/master/doc/files/block_controller.md#%E3%83%96%E3%83%AD%E3%83%83%E3%82%AF%E6%93%8D%E4%BD%9C%E3%81%AE%E5%9F%BA%E6%BA%96%E7%82%B9">形状毎の基準点とそれ以外の座標の表</a></b>によれば基準点は右から2つ目の四角なので、<br>
基準点が盤面の左端 (`x` = `0`)や右端 (`x` = `9`)に位置するような置き方は、ミノが壁にのめり込む形になってしまうので不適切です。<br>
つまり`block_controller.py`が全ての配置パターンを試すとき、基準点の配置を盤面の横幅だけ`for`文で繰り返す実装をするのは無駄で、<br>
関数`getSearchXRange`は`for`文の繰り返しを無駄なく行うための実装に使える、ミノが`x`軸方向に動ける最小の座標と最大の座標を計算してくれます。<br><br>

まずは上と同様に、配置を試してみたいミノの番号とその向きを指定してみてください。

In [ ]:
#------------ここから--------------
minoNo = 
minoDirectionNo = 
#------------ここまで--------------

次に盤面の横幅を取得するため、ゲームの情報が格納された「辞書型」から値を取り出してください。<br>
<span style="color: red; ">**下のセルの空白を正しく埋めた上で、実行してください。**</span>

In [ ]:
# 予め、決められている盤面の横幅を取得しておく
#------------ここから--------------
GameStatus = 
Width = 
#------------ここまで--------------

ミノの基準点が取りうるx座標の最小値・最大値を計算する関数`getSearchXRange`において、ミノの持つ機能を使ってください。<br>
<span style="color: red; ">**下のセルの空白を正しく埋めた上で、実行してください。**</span>

In [ ]:
###################
# ミノの基準点が取りうるx座標の最小値・最大値を計算する
# Shape_class: クラス「Shape」のインスタンス
# direction: テトリミノ回転方向
###################
def getSearchXRange(Shape_class, direction, board_data_width):

        # 指定されたミノのある向きについて、「原点から x,y 両方向に最大何マス占有するのか」を計算する
        # Tips: 最小値は「minX」、最大値は「maxX」という名前の変数に代入してください
        #------------ここから--------------
        
        #------------ここまで--------------

        # 最小値は、ミノの基準点とミノの範囲の左端における距離
        # Tips: …であれば、本来は「xMin = 0 - minX」の方が分かりやすいコードかも
        xMin = -1 * minX
        
        # 最大値は、盤面の右端の座標 (横幅)とミノの範囲の右端における距離
        xMax = (board_data_width - maxX) - 1
        
        return xMin, xMax

以上の実装を、自分で選んだミノとその向きで試してみましょう。<br>
表と照らし合わせて、正しい範囲が出力されていれば穴埋めは成功しています。

In [ ]:
xMinExample, xMaxExample = getSearchXRange(GameStatus["block_info"]["currentShape"]["class"], minoDirectionNo, Width)
print(xMinExample, xMaxExample)

関数`getSearchXRange`をはじめ、「いま盤面に現れたミノに対して全ての配置パターンを試してみる」ための関数が`block_controller.py`には用意されています。<br>
次の例題で使う関数を下のセルにまとめましたので、実行しておいてください。<br>
関数`getBoard`の中で他の関数が使われているので、ひとまず関数`getBoard`の使い方だけを押さえましょう (詳細は例題の中で述べます)。

In [ ]:
import copy


####################################
# direction (回転状態)のテトリミノ座標配列を取得し、それをx,yに配置した場合の2次元座標配列を返す
####################################
def getShapeCoordArray(Shape_class, direction, x, y):

    # direction (回転状態)のテトリミノ座標配列を取得し、それをx,yに配置した場合の2次元座標配列を返す
    coordArray = Shape_class.getCoords(direction, x, y)
    return coordArray


####################################
# 画面ボードデータをコピーして指定座標にテトリミノを配置し落下させた画面ボードとy座標を返す
# board_backboard: 現状画面ボード
# Shape_class: テトリミノ現/予告リスト
# direction: テトリミノ回転方向
# x: テトリミノx座標
####################################
def getBoard(board_backboard, Shape_class, direction, x):
    board = copy.deepcopy(board_backboard)

    # 指定座標から落下させたところにテトリミノを固定しその画面ボードを返す
    _board = dropDown(board, Shape_class, direction, x)
    return _board


####################################
# 指定座標から落下させたところにテトリミノを固定しその画面ボードを返す
# board: 現状画面ボード
# Shape_class: テトリミノ現/予告リスト
# direction: テトリミノ回転方向
# x: テトリミノx座標
####################################
def dropDown(board, Shape_class, direction, x):

    # 画面ボード下限座標として dy 設定
    dy = 22 - 1

    # direction (回転状態)のテトリミノ2次元座標配列を取得し、それをx,yに配置した場合の座標配列を返す
    coordArray = getShapeCoordArray(Shape_class, direction, x, 0)

    # テトリミノ座標配列ごとに...
    for _x, _y in coordArray:
        _yy = 0

        # _yy を一つずつ落とすことによりブロックの落下下限を確認
        # _yy+テトリミノ座標y が 画面下限より上　かつ　(_yy +テトリミノ座標yが画面上限より上 または テトリミノ座標_x,_yy+テトリミノ座標_yのブロックがない)
        while _yy + _y < 22 and (_yy + _y < 0 or board[(_y + _yy) * 10 + _x] == 0):

            #_yy を足していく(下げていく)
            _yy += 1
        _yy -= 1

        # 下限座標 dy /今までの下限より小さい(高い)なら __yy を落下下限として設定
        if _yy < dy:
            dy = _yy
    
    _board = dropDownWithDy(board, Shape_class, direction, x, dy)
    return _board


####################################
# 指定位置にテトリミノを固定する
# board: 現状画面ボード
# Shape_class: テトリミノ現/予告リスト
# direction: テトリミノ回転方向
# x: テトリミノx座標
####################################
def dropDownWithDy(board, Shape_class, direction, x, dy):

    # board コピー
    _board = board
    coordArray = getShapeCoordArray(Shape_class, direction, x, 0)

    # テトリミノ座標配列を順に進める
    for _x, _y in coordArray:

        #x, dy の 画面ボードにブロックを配置して、その画面ボードデータを返す
        _board[(_y + dy) * 10 + _x] = Shape_class.shape
    return _board

それでは、ある盤面にブロックを落としてみましょう。<br>
下のセルは、上の関数定義で「現状画面ボード」という言葉で登場している、いまのゲーム画面を模擬的に作ったものです。<br>
`shapeI`が縦に並んでいて、1ヶ所だけ欠けているような盤面が表示されます。

In [ ]:
board = np.array(
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 1, 1, 1, 1])

# 可視化
visualize_block(board)

この盤面にもう1本`shapeI`ミノを落として、4ライン消しを狙うことを考えてみます。<br><br>

テトリスにおいて、落ちてくるミノに対してプレイヤーができる操作は、<span style="color: red; ">**ミノを横に移動させたり回転させたりする**</span>ことです。<br>
つまりミノを落とすことをゲームで実装したいとき、ミノの基準点の`x`座標と向きを決めてあげる必要があります。<br>
この2つは、`block_controller.py`では変数`strategy`、`block_controller_train.py`では変数`action`という名前の「タプル」で、それぞれ保存されています。<br><br>

因みにこのテトリスの実装では、すぐに作戦を実行するためにブロックを落とす動作をさせたい他に、キーボードでも普通に遊べるようにあるマスだけ落とす動作をさせたいこともあります。<br>
そこで変数`strategy`または`action`の3要素目・4要素目には、「このいずれの動作を行うか」「後者の場合は何マス落とすか」を選択できるようにもなっています。<br><br>

この盤面において、次に盤面に現れてくる`shapeI`ミノで4ライン消しを行うのに適切な値をもった「タプル」を作成してください。<br>
<span style="color: red; ">**下のセルの空白を正しく埋めた上で、実行してください。**</span>

In [ ]:
#------------ここから--------------
strategy = 
#------------ここまで--------------

`shapeI`ミノを狙った場所に落とすための作戦である変数`strategy`を用いて、実際にミノを落としてみましょう。<br>
ここで、`block_controller.py`に定義された「いま盤面に現れたミノに対して全ての配置パターンを試してみる」ための関数を使います。<br>
<span style="color: red; ">**下のセルの空白を正しく埋めた上で、実行してください。**</span>

In [ ]:
# クラス「Shape」のインスタンス化
shape_class = Shape(1)

# 指定座標にテトリミノを配置し落下させた画面ボードを返す
#------------ここから--------------
board_new = 
#------------ここまで--------------

# 可視化
visualize_block(board_new)

- - -

## 3. ミノを落とす方法を決めよう
https://tetris-server-dev.challenge-club.org/training/test/3